In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pickle
import datetime

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/공모전/빅콘테스트 2022/data/user+loan raw data.pickle', 'rb') as f:
    userloan = pickle.load(f)

In [ ]:
user=pd.read_csv('/content/drive/MyDrive/공모전/빅콘테스트 2022/data/user_spec.csv')
loan=pd.read_csv('/content/drive/MyDrive/공모전/빅콘테스트 2022/data/loan_result.csv')
log=pd.read_csv('/content/drive/MyDrive/공모전/빅콘테스트 2022/data/log_data.csv')

# **유저 유니버스 한정**

In [ ]:
log_user=log['user_id'].unique()
len(log_user)   #로그 상 유저아이디

584636

In [ ]:
open_user=log[log['event']=='OpenApp']['user_id'].unique() #openapp을 실행한 유저아이디?
view_user=log[log['event']=='ViewLoanApplyIntro']['user_id'].unique()
start_user=log[log['event']=='StartLoanApply']['user_id'].unique()
end_user=log[log['event']=='EndLoanApply']['user_id'].unique()
print(len(open_user), len(view_user), len(start_user), len(end_user))

418602 356475 326555 302466


In [ ]:
user_id = user['user_id'].unique()
len(user_id)

405213

In [ ]:
#로그 상 유저 목록(584636) > openapp() 유저 목록(418602) > userid 유저 목록(405213)

In [ ]:
print(len(set(user_id).difference(set(log_user)))) #=> user에는 있는데 log에는 없는 user_id

ghost=set(user_id).difference(set(log_user))

34777


In [ ]:
real_user=list(set(user_id).difference(set(ghost))) 
#유저중에서 로그가 없는사람 삭제 => 일전에 가입은 되어있으나 더이상 사용하지 않는 사람

In [ ]:
len(real_user)

370436

In [ ]:
user.sort_values('user_id')

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
1290535,1767402,1,2000.0,1.0,2022-05-03 14:53:33,750.0,20000000.0,OTHERINCOME,NaN,기타,전월세,1000000.0,생활비,0.0,NaN,1.0,NaN
1290534,1031304,1,2000.0,1.0,2022-05-03 14:51:00,750.0,20000000.0,OTHERINCOME,NaN,기타,전월세,0.0,생활비,0.0,NaN,1.0,NaN
971242,1096882,7,1985.0,1.0,2022-05-22 16:42:47,580.0,22000000.0,EARNEDINCOME2,201907.0,계약직,기타가족소유,10000000.0,생활비,0.0,NaN,7.0,46000000.0
1150725,248576,9,1996.0,1.0,2022-05-21 23:41:09,810.0,29000000.0,EARNEDINCOME,202201.0,계약직,기타가족소유,15000000.0,사업자금,0.0,NaN,1.0,NaN
1095661,1521731,11,1997.0,1.0,2022-06-09 11:25:25,670.0,28000000.0,EARNEDINCOME,20210601.0,정규직,기타가족소유,13000000.0,대환대출,0.0,0.0,1.0,14000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691957,1889786,879694,1991.0,0.0,2022-03-31 20:17:31,NaN,39000000.0,EARNEDINCOME,201909.0,정규직,전월세,10000000.0,생활비,NaN,NaN,NaN,NaN
1181092,1774704,879695,1996.0,1.0,2022-05-27 12:50:55,NaN,30000000.0,EARNEDINCOME2,202202.0,정규직,전월세,15000000.0,대환대출,0.0,0.0,NaN,NaN
60175,1856758,879696,1997.0,1.0,2022-03-14 05:40:38,610.0,37000000.0,EARNEDINCOME,202110.0,정규직,전월세,10000000.0,생활비,NaN,NaN,3.0,20000000.0
289906,2149910,879696,1997.0,1.0,2022-03-14 05:23:30,610.0,37000000.0,EARNEDINCOME,202110.0,정규직,전월세,10000000.0,생활비,NaN,NaN,3.0,20000000.0


In [ ]:
user=user[user['user_id'].isin(real_user)]     #리얼 유저들로 실제 유저 데이터프레임 생성

In [ ]:
user_list=user['user_id'].unique().tolist()    #리얼 유저들의 리스트

# **loan+user 합체**

In [ ]:
user

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1.0,27000000.0
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5.0,15000000.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,자가,2000000.0,생활비,0.0,NaN,7.0,344000000.0
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,16000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394210,316356,53701,1996.0,1.0,2022-03-22 10:01:38,730.0,60000000.0,PRIVATEBUSINESS,202105.0,정규직,전월세,15000000.0,사업자금,NaN,NaN,1.0,NaN
1394211,1864587,489900,2000.0,1.0,2022-03-22 14:55:32,590.0,25000000.0,FREELANCER,202106.0,기타,기타가족소유,5000000.0,사업자금,NaN,NaN,NaN,NaN
1394212,1327066,151422,1955.0,1.0,2022-03-22 01:19:24,980.0,20000000.0,OTHERINCOME,NaN,기타,자가,50000000.0,생활비,NaN,NaN,1.0,NaN
1394213,1319606,173524,1983.0,1.0,2022-03-22 07:34:32,750.0,75000000.0,EARNEDINCOME,200908.0,정규직,자가,100000000.0,대환대출,NaN,NaN,8.0,200000000.0


In [ ]:
loan.sort_values('application_id')

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
10972627,1,2022-05-06 11:05:57,62,200,8000000.0,14.8,0.0
10972632,1,2022-05-06 11:05:42,59,150,19000000.0,17.7,0.0
10972631,1,2022-05-06 11:05:43,50,142,18000000.0,19.9,0.0
10972628,1,2022-05-06 11:05:56,38,16,19000000.0,17.1,0.0
10972630,1,2022-05-06 11:05:44,37,206,32000000.0,18.9,0.0
...,...,...,...,...,...,...,...
9732104,2167826,2022-03-25 11:58:01,38,16,22000000.0,16.9,0.0
9732105,2167826,2022-03-25 11:57:51,22,221,13000000.0,17.4,1.0
9732106,2167826,2022-03-25 11:57:49,38,223,22000000.0,16.9,0.0
9732107,2167826,2022-03-25 11:57:49,30,85,5000000.0,18.7,0.0


In [ ]:
 # 신청서 이용한 변수 생성

In [ ]:
app_sum=loan.groupby(['application_id']).sum()  
app_sum

,bank_id,product_id,loan_limit,loan_rate,is_applied
application_id,,,,,
1,284,937,115000000.0,105.5,0.0
2,62,200,15000000.0,9.3,0.0
4,14,411,36000000.0,22.8,0.0
6,492,2489,386000000.0,211.6,0.0
8,150,387,23000000.0,50.3,0.0
...,...,...,...,...,...
2167817,44,149,36000000.0,38.3,1.0
2167819,93,376,78000000.0,49.2,0.0
2167820,109,621,114000000.0,112.1,1.0


In [ ]:
app_sum.describe() #max가 22?

,bank_id,product_id,loan_limit,loan_rate,is_applied
count,968866.000000,968866.000000,9.688660e+05,968866.000000,968866.000000
mean,455.149069,2044.234797,3.173345e+08,177.941578,0.578459
std,516.315776,2338.691472,5.709343e+08,186.396603,0.940610
min,1.000000,1.000000,0.000000e+00,0.000000,0.000000
25%,73.000000,327.000000,2.700000e+07,35.800000,0.000000
50%,242.000000,1084.000000,1.010000e+08,99.900000,0.000000
75%,652.000000,2878.000000,3.570000e+08,261.400000,1.000000
max,2847.000000,12526.000000,2.326270e+11,928.700000,22.000000


In [ ]:
app_sum[app_sum['is_applied']==22] #이런애도 있구나..

,bank_id,product_id,loan_limit,loan_rate,is_applied
application_id,,,,,
1540048,750,3616,278000000.0,254.4,22.0


In [ ]:
loan[loan['application_id']==1540048]

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
12277520,1540048,2022-04-19 12:59:10,58,175,10000000.0,15.8,1.0
12277521,1540048,2022-04-19 12:59:10,32,257,13000000.0,7.2,1.0
12277522,1540048,2022-04-19 12:59:23,22,164,13000000.0,8.0,1.0
12277523,1540048,2022-04-19 12:59:11,36,103,13000000.0,6.7,1.0
12277524,1540048,2022-04-19 12:59:20,38,223,5000000.0,18.4,1.0
12277525,1540048,2022-04-19 12:59:09,38,134,13000000.0,7.5,1.0
12277526,1540048,2022-04-19 12:59:10,31,224,34000000.0,15.6,1.0
12277527,1540048,2022-04-19 12:59:10,33,189,13000000.0,7.8,1.0
12277528,1540048,2022-04-19 12:59:09,27,148,13000000.0,7.0,0.0
12277529,1540048,2022-04-19 12:59:09,10,65,15000000.0,18.5,1.0


In [ ]:
app_mean=loan.groupby(['application_id']).mean()
app_mean

,bank_id,product_id,loan_limit,loan_rate,is_applied
application_id,,,,,
1,47.333333,156.166667,1.916667e+07,17.583333,0.000000
2,62.000000,200.000000,1.500000e+07,9.300000,0.000000
4,7.000000,205.500000,1.800000e+07,11.400000,NaN
6,32.800000,165.933333,2.573333e+07,14.106667,0.000000
8,37.500000,96.750000,7.666667e+06,16.766667,NaN
...,...,...,...,...,...
2167817,22.000000,74.500000,1.800000e+07,19.150000,0.500000
2167819,31.000000,125.333333,2.600000e+07,16.400000,0.000000
2167820,15.571429,88.714286,1.628571e+07,16.014286,0.142857


In [ ]:
main = pd.DataFrame({'app_count':app_mean['bank_id']}) #=> {index : appid}, {값 :신청서 당bank 평균갯수}
main['apply_count'] = app_sum['is_applied']          # 신청서 당 apply한 대출 개수 총합
main['loanlimit_mean'] = app_mean['loan_limit']       # 대출한도의 평균
main['rate_mean'] = app_mean['loan_rate']        # 대출이율의 평균
main.describe()

,app_count,apply_count,loanlimit_mean,rate_mean
count,968866.000000,968866.000000,9.687270e+05,968727.000000
mean,32.058751,0.578459,1.662837e+07,14.022585
std,9.702650,0.940610,1.683227e+07,2.999882
min,1.000000,0.000000,0.000000e+00,1.500000
25%,28.000000,0.000000,9.769231e+06,11.989655
50%,32.600000,0.000000,1.490909e+07,14.200000
75%,36.425000,1.000000,2.166667e+07,16.194444
max,64.000000,22.000000,5.057109e+09,20.000000


In [ ]:
main

,app_count,apply_count,loanlimit_mean,rate_mean
application_id,,,,
1,47.333333,0.0,1.916667e+07,17.583333
2,62.000000,0.0,1.500000e+07,9.300000
4,7.000000,0.0,1.800000e+07,11.400000
6,32.800000,0.0,2.573333e+07,14.106667
8,37.500000,0.0,7.666667e+06,16.766667
...,...,...,...,...
2167817,22.000000,1.0,1.800000e+07,19.150000
2167819,31.000000,0.0,2.600000e+07,16.400000
2167820,15.571429,1.0,1.628571e+07,16.014286


In [ ]:
main=main[main.index.isin(user['application_id'].unique())] #user에 있는애들로 loan 한정

In [ ]:
#user+loan
df=pd.merge(user,main,how='left',on='application_id')
df

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,app_count,apply_count,loanlimit_mean,rate_mean
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,...,1000000.0,기타,0.0,NaN,4.0,162000000.0,NaN,NaN,NaN,NaN
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,...,30000000.0,대환대출,0.0,NaN,1.0,27000000.0,32.916667,0.0,2.375000e+07,11.541667
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,...,10000000.0,생활비,0.0,NaN,5.0,15000000.0,31.545455,0.0,2.009091e+07,14.563636
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,...,2000000.0,생활비,0.0,NaN,7.0,344000000.0,37.360000,0.0,1.308000e+07,13.180000
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,...,5000000.0,생활비,0.0,0.0,1.0,16000000.0,56.000000,0.0,9.500000e+06,16.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355401,316356,53701,1996.0,1.0,2022-03-22 10:01:38,730.0,60000000.0,PRIVATEBUSINESS,202105.0,정규직,...,15000000.0,사업자금,NaN,NaN,1.0,NaN,31.555556,3.0,1.644444e+07,12.111111
1355402,1864587,489900,2000.0,1.0,2022-03-22 14:55:32,590.0,25000000.0,FREELANCER,202106.0,기타,...,5000000.0,사업자금,NaN,NaN,NaN,NaN,18.250000,3.0,1.375000e+07,16.600000
1355403,1327066,151422,1955.0,1.0,2022-03-22 01:19:24,980.0,20000000.0,OTHERINCOME,NaN,기타,...,50000000.0,생활비,NaN,NaN,1.0,NaN,34.600000,0.0,3.720000e+07,10.490000
1355404,1319606,173524,1983.0,1.0,2022-03-22 07:34:32,750.0,75000000.0,EARNEDINCOME,200908.0,정규직,...,100000000.0,대환대출,NaN,NaN,8.0,200000000.0,32.666667,0.0,1.333333e+07,14.116667


In [ ]:
df.isna().sum()

application_id                               0
user_id                                      0
birth_year                               12532
gender                                   12532
insert_time                                  0
credit_score                             74719
yearly_income                               90
income_type                                 85
company_enter_month                     164637
employment_type                             85
houseown_type                               85
desired_amount                              85
purpose                                     85
personal_rehabilitation_yn              566342
personal_rehabilitation_complete_yn    1165358
existing_loan_cnt                       164428
existing_loan_amt                       279304
app_count                               412633
apply_count                             412633
loanlimit_mean                          412659
rate_mean                               412659
dtype: int64

In [ ]:
df.to_csv('/content/drive/MyDrive/공모전/빅콘테스트 2022/data/클러/clu_df.csv')

# **log 시계열 클러스터 변수 생성**

log 하루당 사용횟수로 시계열 생성 -> 시계열 클러스터링 통해 빈도 군집 생성
부차적 기능 시계열 / 메인기능 시계열

In [ ]:
log

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,Android,3.8.4,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,iOS,3.6.1,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,iOS,3.6.1,2022-03-25
...,...,...,...,...,...,...
17843988,242442,StartLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
17843989,242442,EndLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
17843990,593062,OpenApp,2022-05-27 16:19:10,Android,3.13.0,2022-05-27
17843991,593062,Login,2022-05-27 16:19:15,Android,3.13.0,2022-05-27


In [ ]:
log_df = pd.DataFrame({'date':pd.date_range('2022-03-01','2022-06-30', freq='D')})

In [ ]:
log_df

,date
0,2022-03-01
1,2022-03-02
2,2022-03-03
3,2022-03-04
4,2022-03-05
...,...
117,2022-06-26
118,2022-06-27
119,2022-06-28
120,2022-06-29


In [ ]:
log['date_cd']=pd.to_datetime(log['date_cd'])  # datecd datetime 화

In [ ]:
user_log=log[log['user_id'].isin(user_list)] # 리얼유저들에 해당하는 로그 추출

In [ ]:
log['event'].unique()

array(['StartLoanApply', 'ViewLoanApplyIntro', 'EndLoanApply', 'OpenApp',
       'UseLoanManage', 'UsePrepayCalc', 'Login',
       'CompleteIDCertification', 'UseDSRCalc', 'SignUp', 'GetCreditInfo'],
      dtype=object)

In [ ]:
mai = ['StartLoanApply', 'ViewLoanApplyIntro', 'EndLoanApply'] # 대출신청 주요기능
sub = ['UseLoanManage', 'UsePrepayCalc', 'UseDSRCalc'] # 대출신청 부가기능

In [ ]:
log_g=user_log.groupby(['user_id','date_cd']).count()['event'] #예시

In [ ]:
log_all = log_g.reset_index()
log_all  #예시

,user_id,date_cd,event
0,1,2022-05-03,2
1,1,2022-06-16,3
2,7,2022-05-22,1
3,9,2022-05-21,3
4,11,2022-03-24,12
...,...,...,...
2155603,879693,2022-06-27,4
2155604,879693,2022-06-29,1
2155605,879694,2022-03-31,6
2155606,879695,2022-05-27,4


In [ ]:
m = user_log[user_log['event'].isin(mai)]       #각각 main과 sub기능들 날짜별로 카운트
s = user_log[user_log['event'].isin(sub)]
log_main=m.groupby(['user_id','date_cd']).count()['event']
log_sub=s.groupby(['user_id','date_cd']).count()['event'] 

In [ ]:
log_main = log_main.reset_index()    # groupby로 인한 이중인덱스 해제
log_sub = log_sub.reset_index()

In [ ]:
log_main.to_csv('/content/drive/MyDrive/공모전/빅콘테스트 2022/data/클러/log_main.csv')
log_sub.to_csv('/content/drive/MyDrive/공모전/빅콘테스트 2022/data/클러/log_sub.csv')

In [ ]:
from datetime import datetime, timedelta   #datetime range str형으로 뽑아내는 함수

def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

In [ ]:
columns = pd.date_range('2022-03-01','2022-07-03', freq='W').astype(str)   # 일주일 기점으로 칼럼 생성
columns 

Index(['2022-03-06', '2022-03-13', '2022-03-20', '2022-03-27', '2022-04-03',
       '2022-04-10', '2022-04-17', '2022-04-24', '2022-05-01', '2022-05-08',
       '2022-05-15', '2022-05-22', '2022-05-29', '2022-06-05', '2022-06-12',
       '2022-06-19', '2022-06-26', '2022-07-03'],
      dtype='object')

In [ ]:
cluster_sub=pd.DataFrame({'user':user_list})   # df_sub 생성
cluster_sub[columns]=0
cluster_sub.index = cluster_sub.user
cluster_sub.pop('user')

user
118218    118218
553686    553686
59516      59516
167320    167320
33400      33400
           ...  
762698    762698
53701      53701
489900    489900
151422    151422
173524    173524
Name: user, Length: 370436, dtype: int64

In [ ]:
cluster_sub

,2022-03-06,2022-03-13,2022-03-20,2022-03-27,2022-04-03,2022-04-10,2022-04-17,2022-04-24,2022-05-01,2022-05-08,2022-05-15,2022-05-22,2022-05-29,2022-06-05,2022-06-12,2022-06-19,2022-06-26,2022-07-03
user,,,,,,,,,,,,,,,,,,
118218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
553686,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59516,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
167320,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762698,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53701,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
489900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
def ch_date(i):                                  # log_sub의 일 기준 date_cd 주 기준으로 변환
  if (i>='2022-03-01')&(i<='2022-03-06'):
    return '2022-03-06'
  elif (i>='2022-03-07')&(i<='2022-03-13'):
    return '2022-03-13'
  elif (i>='2022-03-14')&(i<='2022-03-20'):
    return '2022-03-20'
  elif (i>='2022-03-21')&(i<='2022-03-27'):
    return '2022-03-27'
  elif (i>='2022-03-28')&(i<='2022-04-03'):
    return '2022-04-03'
  elif (i>='2022-04-04')&(i<='2022-04-10'):
    return '2022-04-10'
  elif (i>='2022-04-11')&(i<='2022-04-17'):
    return '2022-04-17'
  elif (i>='2022-04-18')&(i<='2022-04-24'):
    return '2022-04-24'
  elif (i>='2022-04-25')&(i<='2022-05-01'):
    return '2022-05-01'
  elif (i>='2022-05-02')&(i<='2022-05-08'):
    return '2022-05-08'
  elif (i>='2022-05-09')&(i<='2022-05-15'):
    return '2022-05-15'
  elif (i>='2022-05-16')&(i<='2022-05-22'):
    return '2022-05-22'
  elif (i>='2022-05-23')&(i<='2022-05-29'):
    return '2022-05-29'
  elif (i>='2022-05-30')&(i<='2022-06-05'):
    return '2022-06-05'
  elif (i>='2022-06-06')&(i<='2022-06-12'):
    return '2022-06-12'
  elif (i>='2022-06-13')&(i<='2022-06-19'):
    return '2022-06-19'
  elif (i>='2022-06-20')&(i<='2022-06-26'):
    return '2022-06-26'
  elif (i>='2022-06-27')&(i<='2022-07-03'):
    return '2022-07-03'



In [ ]:
log_sub

,user_id,date_cd,event
0,1,2022-06-16,1
1,11,2022-03-24,4
2,11,2022-04-20,6
3,12,2022-05-01,1
4,17,2022-03-07,1
...,...,...,...
826685,879693,2022-05-13,1
826686,879693,2022-05-17,1
826687,879693,2022-05-20,1
826688,879693,2022-06-14,2


In [ ]:
log_sub['date_cd'] = log_sub['date_cd'].astype(str) #비교 용이하게 하기 위해 str형으로 변환

In [ ]:
log_sub['date_cd']= list(map(ch_date,log_sub['date_cd'])) #ch_date 적용

In [ ]:
log_sub = log_sub.groupby(['user_id','date_cd']).sum().reset_index() #주 단위로 변경하며 일치하게 되는 값들 sum
log_sub

,user_id,date_cd,event
0,1,2022-06-19,1
1,11,2022-03-27,4
2,11,2022-04-24,6
3,12,2022-05-01,1
4,17,2022-03-13,1
...,...,...,...
611619,879692,2022-05-01,1
611620,879693,2022-05-15,1
611621,879693,2022-05-22,2
611622,879693,2022-06-19,2


In [ ]:
def log_time(i):   # log sub의 1행을 -> 인덱스, 2행 -> 열로 가지는 df_sub 자리에 event 넣기
  cluster_sub.at[log_sub.iat[i,0],log_sub.iat[i,1]]=log_sub.iat[i,2] 

In [ ]:
t=list(map(log_time,range(len(log_sub))))

In [ ]:
cluster_sub

,2022-03-06,2022-03-13,2022-03-20,2022-03-27,2022-04-03,2022-04-10,2022-04-17,2022-04-24,2022-05-01,2022-05-08,2022-05-15,2022-05-22,2022-05-29,2022-06-05,2022-06-12,2022-06-19,2022-06-26,2022-07-03
user,,,,,,,,,,,,,,,,,,
118218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
553686,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0
59516,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0
167320,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0
33400,0,0,0,0,0,0,0,0,0,4,0,0,0,5,2,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762698,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53701,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
489900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#main도 동일하게 진행
cluster_main=pd.DataFrame({'user':user_list})   # df_sub 생성
cluster_main[columns]=0
cluster_main.index = cluster_main.user
cluster_main.pop('user')
log_main['date_cd'] = log_main['date_cd'].astype(str) #비교 용이하게 하기 위해 str형으로 변환
log_main['date_cd']= list(map(ch_date,log_main['date_cd'])) #ch_date 적용
log_main = log_main.groupby(['user_id','date_cd']).sum().reset_index() #주 단위로 변경하며 일치하게 되는 값들 sum
def log_time(i):   # log sub의 1행을 -> 인덱스, 2행 -> 열로 가지는 df_sub 자리에 event 넣기
  cluster_main.at[log_main.iat[i,0],log_main.iat[i,1]]=log_main.iat[i,2] 
t=list(map(log_time,range(len(log_main))))
cluster_main

,2022-03-06,2022-03-13,2022-03-20,2022-03-27,2022-04-03,2022-04-10,2022-04-17,2022-04-24,2022-05-01,2022-05-08,2022-05-15,2022-05-22,2022-05-29,2022-06-05,2022-06-12,2022-06-19,2022-06-26,2022-07-03
user,,,,,,,,,,,,,,,,,,
118218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
553686,0,0,0,0,0,0,0,0,0,0,0,3,0,0,3,0,0,3
59516,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,4,0
167320,0,0,0,0,0,3,0,0,3,3,3,2,0,0,9,0,0,0
33400,0,0,0,0,0,0,0,0,0,0,0,0,0,16,6,11,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762698,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53701,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
489900,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
cluster_sub.to_csv('/content/drive/MyDrive/공모전/빅콘테스트 2022/data/클러/cluster_sub.csv')

In [ ]:
cluster_main.to_csv('/content/drive/MyDrive/공모전/빅콘테스트 2022/data/클러/cluster_main.csv')